In [1]:
import pandas as pd

import pathlib as pl

import os

In [2]:
import shutil

In [3]:
data_folder = pl.Path("data")

labels_df = pd.read_csv(data_folder / "train.csv")

In [4]:
labels_df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [5]:
len(labels_df)

7095

In [6]:
labels_df.ClassId.value_counts()

3    5150
1     897
4     801
2     247
Name: ClassId, dtype: int64

In [7]:
labels_df.ImageId.value_counts()

db4867ee8.jpg    3
ef24da2ba.jpg    3
2930b4f52.jpg    2
c63e60886.jpg    2
bab5271b6.jpg    2
                ..
542084ea9.jpg    1
10b23eabe.jpg    1
c2bc67c36.jpg    1
fdab86eb7.jpg    1
8aaf235ef.jpg    1
Name: ImageId, Length: 6666, dtype: int64

In [8]:
train_image_folder = data_folder / "train_images"
test_image_folder = data_folder / "test_images"

In [9]:
train_image_files = [
    f 
    for f in os.listdir(train_image_folder)
    if f.endswith(".jpg")
]

In [10]:
len(train_image_files)

12568

In [11]:
test_image_files = [
    f 
    for f in os.listdir(test_image_folder)
    if f.endswith(".jpg")
]

In [12]:
len(test_image_files)

5506

In [13]:
severstal_binary_folder = data_folder / "severstal-binary"

In [14]:
if severstal_binary_folder.exists():
    shutil.rmtree(severstal_binary_folder)

In [15]:
os.makedirs(severstal_binary_folder / "train" / "good")
os.makedirs(severstal_binary_folder / "train" / "bad")

os.makedirs(severstal_binary_folder / "test" / "good")
os.makedirs(severstal_binary_folder / "test" / "bad")

In [16]:
take_every_nth = 3  # to reduce disk space

In [17]:
bad_images = labels_df.ImageId.values

for i, image in enumerate(train_image_files):
    if i % take_every_nth == 0:
        if image in bad_images:
            shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "bad" /image)
        else:
            shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "good" /image)

In [18]:
for i, image in enumerate(test_image_files):
    if i % take_every_nth == 0:
        if image in bad_images:
            shutil.copy(test_image_folder / image, severstal_binary_folder / "test" / "bad" /image)
        else:
            shutil.copy(test_image_folder / image, severstal_binary_folder / "test" / "good" /image)